In [2]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`


import org.apache.spark.sql._
import org.apache.spark.sql.functions._


//Reducir numero logs
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


//Circe para trabajar con JSON
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._


//Sttp para realizar request
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._
import sttp.client.basicRequest

//For adding extra dependenies
import $ivy.`org.typelevel::cats-core:1.6.0`

//Plotly
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import $ivy.$                                  



import org.apache.spark.sql._

import org.apache.spark.sql.functions._


//Reducir numero logs

import org.apache.log4j.{Level, Logger}

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._


//Sttp para realizar request

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._

import sttp.client.basicRequest

//For adding extra dependenies

import $ivy.$                               

//Plotly

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

In [3]:
//Iniciamos session de Spark
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
import spark.implicits._

//Crear URL
def createUri(category: String, widget: String, start_date: String, end_date: String, time_trunc: String, lang: String, 
              geo_trunc: String = "", geo_limit: String = "", geo_ids: String = ""): String = {
  val baseUrl = s"https://apidatos.ree.es/$lang/datos/$category/$widget"
  
// Crear un mapa con los parámetros obligatorios y opcionales
  val params = Map(
    "start_date" -> start_date,
    "end_date" -> end_date,
    "time_trunc" -> time_trunc,
    "geo_trunc" -> geo_trunc,
    "geo_limit" -> geo_limit,
    "geo_ids" -> geo_ids
  ).filter { case (_, v) => v.nonEmpty } // Filtrar los parámetros que no están vacíos

//Especifico el orden concreto de los paramtros ya que, Scala map no te garantiza el orden
  val orderedKeys = Seq("start_date", "end_date", "time_trunc", "geo_trunc", "geo_limit", "geo_ids")

// Unir los parámetros en una cadena
  val queryString = orderedKeys.flatMap(key => params.get(key).map(value => s"$key=$value")).mkString("&")
  
  s"$baseUrl?$queryString"
}

//Lanzar llamada y obtener guardar datos
def getDataApi(apiUrl: String): Either[String, String] = {
 
    val newRequest = basicRequest.get(uri"$apiUrl")
    val response = newRequest.send()
  
  response.body match {
    case Right(body) => 
      Right(body)
      
    case Left(error) => 
      Left(s"Error fetching data: $error")
  }
}

// Transformar response a DataFrame
def responseToDF(response: Either[String, String]): DataFrame = {
    response match {
    case Right(body) => 
        val rdd = spark.sparkContext.parallelize(Seq(body))
        spark.read.json(rdd)
    case Left(error) => 
        println(error)
        spark.emptyDataFrame
    }
}



Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@3a86248a
import spark.implicits._

//Crear URL

defined function createUri
defined function getDataApi
defined function responseToDF

### Derinición rango de uris

In [5]:
import java.time.format.DateTimeFormatter
import java.time.LocalDate

val category = "demanda"
val widget = "demanda-tiempo-real"
val time_trunc = "hour"
val lang = "es"

def crearRango(primerAño: Int, ultimoAño: Int): Seq[String]  = {
val rango = for {
  year <- primerAño to ultimoAño
  month <- 1 to 12
} yield {
  val init = LocalDate.of(year, month, 1).toString + "T00:00"
  val end = LocalDate.of(year, month, 1).plusMonths(1).minusDays(1).toString + "T23:59"
  (init, end)
}

rango.map {
    case (init, end) => createUri(category, widget, init, end, time_trunc,lang)
    }
}
val urisDemanda = crearRango(2014,2024).dropRight(5)

import java.time.format.DateTimeFormatter

import java.time.LocalDate


category: String = "demanda"
widget: String = "demanda-tiempo-real"
time_trunc: String = "hour"
lang: String = "es"
defined function crearRango
urisDemanda: Seq[String] = Vector(
  "https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real?start_date=2014-01-01T00:00&end_date=2014-01-31T23:59&time_trunc=hour",
  "https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real?start_date=2014-02-01T00:00&end_date=2014-02-28T23:59&time_trunc=hour",
  "https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real?start_date=2014-03-01T00:00&end_date=2014-03-31T23:59&time_trunc=hour",
  "https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real?start_date=2014-04-01T00:00&end_date=2014-04-30T23:59&time_trunc=hour",
  "https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real?start_date=2014-05-01T00:00&end_date=2014-05-31T23:59&time_trunc=hour",
  "https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real?start

In [6]:
val listDemandaJson = urisDemanda.map(getDataApi(_))

listDemandaJson: Seq[Either[String, String]] = Vector(
  Right(
    "{\"data\":{\"type\":\"Demanda peninsular en tiempo real\",\"id\":\"dem15\",\"attributes\":{\"title\":\"Demanda peninsular en tiempo real\",\"last-update\":\"2016-12-20T11:50:25.000+01:00\",\"description\":null},\"meta\":{\"cache-control\":{\"cache\":\"MISS\"}}},\"included\":[{\"type\":\"Demanda real\",\"id\":\"1293\",\"groupId\":null,\"attributes\":{\"title\":\"Demanda real\",\"description\":\"Demanda real\",\"color\":\"#ffea00\",\"type\":null,\"magnitude\":null,\"composite\":false,\"last-update\":\"2016-12-20T11:50:25.000+01:00\",\"values\":[{\"value\":24546,\"percentage\":0.33566261435584666,\"datetime\":\"2014-01-01T00:00:00.000+01:00\"},{\"value\":24309,\"percentage\":0.33504238164151334,\"datetime\":\"2014-01-01T00:10:00.000+01:00\"},{\"value\":24348,\"percentage\":0.33662846161290766,\"datetime\":\"2014-01-01T00:20:00.000+01:00\"},{\"value\":24321,\"percentage\":0.3372857380595773,\"datetime\":\"2014-01-01T00:30

In [7]:
listDemandaJson.size

res6: Int = 127

In [ ]:
def transformDataFrame(dFrameDemanda: DataFrame): DataFrame = {
    // Definir los tipos de energía de bajas emisiones
    val transformedDfDemanda = dFrameDemanda
        .withColumn("included", explode($"included"))
        .withColumn("TipoDemanda", $"included.type")
        .withColumn("Values", explode($"included.attributes.values"))
        .select(
            $"Values.datetime".cast("timestamp").as("Fecha"),
            $"TipoDemanda",
            $"Values.value".as("Valor"),
            $"Values.percentage".as("Porcentaje")
        )
    
    val pivotedDfDemanda = transformedDfDemanda     
        .groupBy("Fecha")
        .pivot("TipoDemanda")
        .agg(
            first("Valor").as("Valor"),
            first("Porcentaje").as("Porcentaje")
        )
        .withColumnRenamed("Demanda prevista_Valor", "Valor_previsto")
        .withColumnRenamed("Demanda prevista_Porcentaje", "Porcentaje_previsto")
        .withColumnRenamed("Demanda programada_Valor", "Valor_programado")
        .withColumnRenamed("Demanda programada_Porcentaje", "Porcentaje_programado")
        .withColumnRenamed("Demanda real_Valor", "Valor_real")
        .withColumnRenamed("Demanda real_Porcentaje", "Porcentaje_real")
    
    pivotedDfDemanda
}

// Reducir primero todos los DataFrames y luego aplicar la transformación

val listModelsDemanda = listDemandaJson
    .map(json => transformDataFrame(responseToDF(json)))
    .reduce(_ union _)


json at cmd2.sc:52

16 / 16

pivot at cmd7.sc:16

16 / 16

pivot at cmd7.sc:16

...

16 / 16

pivot at cmd7.sc:16

16 / 16

pivot at cmd7.sc:16 
 (kill)

27 + 17 / 200

In [ ]:
listModelsDemanda.coalesce(1)
    .write
      .option("header", "true")
      .csv("data/modelosCsv/datasetDemanda24.csv")